In [1]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime

In [2]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [3]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string


def find_qrt_date(content):
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None:
        qtr_match = qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    return remove_multiple_spaces(str(qtr_match.group()))

In [4]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC__sec_filing_links.xlsx")
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2010-11-04,2010-09-30,https://www.sec.gov/Archives/edgar/data/128775...
1,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2011-03-01,2010-12-31,https://www.sec.gov/Archives/edgar/data/128775...
2,10-Q,Quarterly report [Sections 13 or 15(d)],2011-05-03,2011-03-31,https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],2011-08-04,2011-06-30,https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2011-11-08,2011-09-30,https://www.sec.gov/Archives/edgar/data/128775...


In [5]:
# drops all the amendment filing
filing_links = filing_links.drop(filing_links[filing_links['Form description'].str.contains(
    'amendment', case=False)].index).reset_index(drop=True)
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],2010-11-04,2010-09-30,https://www.sec.gov/Archives/edgar/data/128775...
1,10-K,"Annual report [Section 13 and 15(d), not S-K I...",2011-03-01,2010-12-31,https://www.sec.gov/Archives/edgar/data/128775...
2,10-Q,Quarterly report [Sections 13 or 15(d)],2011-05-03,2011-03-31,https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],2011-08-04,2011-06-30,https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],2011-11-08,2011-09-30,https://www.sec.gov/Archives/edgar/data/128775...


In [6]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head()

,Form type,Form description,Filing date,Reporting date,Filings URL
0,10-Q,Quarterly report [Sections 13 or 15(d)],"November 04, 2010","September 30, 2010",https://www.sec.gov/Archives/edgar/data/128775...
1,10-K,"Annual report [Section 13 and 15(d), not S-K I...","March 01, 2011","December 31, 2010",https://www.sec.gov/Archives/edgar/data/128775...
2,10-Q,Quarterly report [Sections 13 or 15(d)],"May 03, 2011","March 31, 2011",https://www.sec.gov/Archives/edgar/data/128775...
3,10-Q,Quarterly report [Sections 13 or 15(d)],"August 04, 2011","June 30, 2011",https://www.sec.gov/Archives/edgar/data/128775...
4,10-Q,Quarterly report [Sections 13 or 15(d)],"November 08, 2011","September 30, 2011",https://www.sec.gov/Archives/edgar/data/128775...


In [7]:
filing_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Form type         52 non-null     object
 1   Form description  52 non-null     object
 2   Filing date       52 non-null     object
 3   Reporting date    52 non-null     object
 4   Filings URL       52 non-null     object
dtypes: object(5)
memory usage: 2.2+ KB


In [8]:
# def extract_tables(soup_content, qtr_date):
#     consolidated_schedule_regex = re.compile(
#         r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
#     date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

#     master_table = None
#     all_tags = soup_content.find_all(True)
#     count = 0

#     for tag in soup_content.find_all(text=consolidated_schedule_regex):
#         next_line_text = tag.next.text.strip()
#         date_str = re.search(date_regex_pattern, next_line_text)

#         if date_str is None:
#             next_line_text = tag.find_next(
#                 text=re.compile(date_regex_pattern)).text
#             date_str = re.search(date_regex_pattern, next_line_text)

#         if date_str is None:
#             next_line = tag.next.text.strip()
#             date_str = re.search(date_regex_pattern, next_line)

#         if date_str is not None:
#             date_str = str(date_str.group(1))
#             date_str = unicodedata.normalize('NFKD', date_str)

#             if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
#                 count += 1
#                 html_table = tag.find_next('table')
#                 while html_table:
#                     new_table = pd.read_html(
#                         html_table.prettify(), skiprows=0, flavor='bs4')[0]
#                     new_table = new_table.applymap(lambda x: unicodedata.normalize(
#                         'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
#                     new_table = new_table.replace(
#                         r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
#                     new_table = new_table.dropna(how='all', axis=0)

#                     if master_table is None:
#                         master_table = new_table
#                     else:
#                         master_table = pd.concat(
#                             [master_table, new_table], ignore_index=True)

#                     # print(master_table)
#                     if html_table.find_next(text=consolidated_schedule_regex):
#                         next_line_text = tag.find_next(
#                             text=re.compile(date_regex_pattern)).text
#                         date_str2 = re.search(
#                             date_regex_pattern, next_line_text)
#                         if date_str2.group(1) == date_str:
#                             break
#                     html_table = html_table.find_next('table')

#     master_table = master_table.applymap(
#         lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
#     master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
#         r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
#     return master_table


# master_table = extract_tables(content, qtr_date)
# master_table

In [9]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775023000036/arcc-20230630.htm'
# qtr_date = 'June 30, 2023'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775019000013/arccq1-1910q.htm'
# qtr_date = 'March 31, 2019'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000104746911001575/a2202281z10-k.htm'
# qtr_date = 'December 31, 2010'
url = 'https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm'
qtr_date = 'September 30, 2010'
# url = 'https://www.sec.gov/Archives/edgar/data/1287750/000128775018000015/arccq1-1810q.htm'
# qtr_date = 'March 31, 2018'
response = requests.get(url, headers=headers)
content = parse_and_trim(response.content, 'HTML')


def extract_tables(soup_content, qtr_date):
    consolidated_schedule_regex = re.compile(
        r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'

    master_table = None
    all_tags = soup_content.find_all(True)
    count = 0

    for tag in soup_content.find_all(text=consolidated_schedule_regex):
        # print(len(soup_content.find_all(text=consolidated_schedule_regex)))
        next_line_text = tag.next.text.strip()
        date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line_text = tag.find_next(
                text=re.compile(date_regex_pattern)).text
            date_str = re.search(date_regex_pattern, next_line_text)

        if date_str is None:
            next_line = tag.next.text.strip()
            date_str = re.search(date_regex_pattern, next_line)

        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)

            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                count += 1
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

                    if html_table.find(text=re.compile(r'Other than', re.IGNORECASE)):
                        break
                    elif html_table.find_next(text='Derivative Instruments'):
                        break
                    elif html_table.find_next(text=consolidated_schedule_regex):
                        next_line_text = tag.find_next(
                            text=re.compile(date_regex_pattern)).text
                        date_str2 = re.search(
                            date_regex_pattern, next_line_text)
                        if date_str2.group(1) == date_str:
                            break
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    return master_table

In [10]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    # print('1: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    # print('2: ' + str(soi_table_data_df.shape))

    # drops all the rows that contains header
    soi_table_data_df = soi_table_data_df[soi_table_data_df[soi_table_data_df.columns[0]]
                                          != soi_table_data_df.columns[0]]
    # print('3: ' + str(soi_table_data_df.shape))

    # Drop rows with only two non-null values becuase all the subtotal contain 2 value only
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    # print('4: ' + str(soi_table_data_df.shape))

    # replace all the - in the data table with 0
    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)

    # fix the all the nan value column , Amortized Cost, Fair Value
    columns_to_fill = ['Amortized Cost', 'Fair Value']
    for col in columns_to_fill:
        col_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc(col.replace(' ', ''))
        next_col_index = col_index + 1
        for i in range(len(soi_table_data_df)):
            current_value = soi_table_data_df.iat[i, col_index]
            if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                next_valid_index = next((j for j, v in enumerate(
                    soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                if next_valid_index is not None:
                    next_value = soi_table_data_df.iat[i, next_valid_index]
                    soi_table_data_df.iat[i, col_index] = next_value
                    soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # drops everything after FairValue
    if 'FairValue' in soi_table_data_df.columns.str.replace(' ', ''):
        start_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc('FairValue')
        soi_table_data_df = soi_table_data_df.iloc[:, :start_index+1]

    # Drop rows with only two non-null values this one recheacks
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)

    # Forward fill the first two columns
    col_indices = [0, 1]
    soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
        method='ffill')
    # print('7: ' + str(soi_table_data_df.shape))

    # Drop rows with all missing values
    soi_table_df = soi_table_df.dropna(how='all')
    # print('5: ' + str(soi_table_data_df.shape))

    # Drop columns with all missing values
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    # print('6: ' + str(soi_table_data_df.shape))

    # fill all the nan with 0
    soi_table_data_df = soi_table_data_df.fillna(0)

    # converts to number
    # cols_to_convert = ['Shares/Units', 'Principal',
    #                    'Amortized Cost', 'Fair Value']
    # for col in cols_to_convert:
    #     if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
    #         col_index = soi_table_data_df.columns.str.replace(
    #             ' ', '').get_loc(col.replace(' ', ''))
    #         soi_table_data_df.iloc[:, col_index] = pd.to_numeric(
    #             soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)

    cols_to_convert = ['Shares/Units', 'Principal',
                       'Amortized Cost', 'Fair Value']
    for col in cols_to_convert:
        if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
            col_index = soi_table_data_df.columns.str.replace(
                ' ', '').get_loc(col.replace(' ', ''))
            converted_data = pd.to_numeric(
                soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)
            soi_table_data_df[soi_table_data_df.columns[col_index]
                              ] = converted_data

    soi_table_data_df = soi_table_data_df.reset_index(drop=True)

    # keeping the total for now to check if the total is right
    # Drop rows based on regex pattern (e.g., 'subtotal' or 'total')
    # pattern = r'^([Ss]ubtotal)|([Tt]otal)'
    # mask = soi_table_data_df.apply(lambda row: row.astype(
    #     str).str.contains(pattern, case=False, na=False)).any(axis=1)
    # soi_table_data_df = soi_table_data_df[~mask]
    # # print('4: ' + str(soi_table_data_df.shape))

    # we dont need this here because we are dropping subtotal before
    # Drop rows labeled as subtotals
    # subtotal_rows = soi_table_data_df[soi_table_data_df['Company (1)'].str.replace(' ', '').str.contains(
    #     'subtotal', case=False, na=False)]
    # soi_table_data_df = soi_table_data_df[~soi_table_data_df.index.isin(
    #     subtotal_rows.index)]
    # print('3: ' + str(soi_table_data_df.shape))

    # we dont need this here because we are dropping subtotal before
    # columns_to_drop = []
    # for column in soi_table_data_df.columns:
    #     # Check for NaN values in the column
    #     # Use .item() to get a single boolean value
    #     if soi_table_data_df[column].isna().any().item():
    #         columns_to_drop.append(column)
    # soi_table_data_df.drop(columns=columns_to_drop, inplace=True)

    # print('Final: ' + str(soi_table_data_df.shape))

    return soi_table_data_df

In [11]:
count = 0
path = '/Users/fuadhassan/Desktop/BDC_RA/ARCC/ARCC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(qtr_date)
    print('Starting file # ', count)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    print('Getting content done for # ', count)
    master_table = extract_tables(content, qtr_date)
    print("Done creating master_table # ", count)
    processed_table_ = process_table(
        master_table, qtr_date.replace(',', ''))
    processed_table_.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    processed_table_.to_csv(
        '../csv_file/'+qtr_date.replace(',', '')+'.csv')
    print("Done processed_table # ", count)
    count += 1
    writer.save()
writer.close()

September 30, 2010
Starting file #  0
Getting content done for #  0
Done creating master_table #  0
Done processed_table #  0
December 31, 2010
Starting file #  1
Getting content done for #  1
Done creating master_table #  1
Done processed_table #  1
March 31, 2011
Starting file #  2
Getting content done for #  2
Done creating master_table #  2
Done processed_table #  2
June 30, 2011
Starting file #  3
Getting content done for #  3
Done creating master_table #  3
Done processed_table #  3
September 30, 2011
Starting file #  4
Getting content done for #  4
Done creating master_table #  4
Done processed_table #  4
December 31, 2011
Starting file #  5
Getting content done for #  5
Done creating master_table #  5
Done processed_table #  5
March 31, 2012
Starting file #  6
Getting content done for #  6
Done creating master_table #  6
Done processed_table #  6
June 30, 2012
Starting file #  7
Getting content done for #  7
Done creating master_table #  7
Done processed_table #  7
September 30

In [12]:
# testing
url = filing_links.iloc[0]['Filings URL']
date = filing_links.iloc[0]['Reporting date']
url, date
# response = requests.get(url, headers=headers)
# content = parse_and_trim(response.content, 'HTML')
# master_table = extract_tables(content, date)
# process_table_ = process_table(master_table, "")
# process_table_.to_excel("example.xlsx")
# process_table_.to_csv('example.csv')
# process_table_

('https://www.sec.gov/Archives/edgar/data/1287750/000110465910055721/a10-17362_110q.htm',
 'September 30, 2010')

In [13]:
print('done')

done


In [14]:
master_table

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,Company (1),NaN,Business Description,NaN,Investment,NaN,Coupon (3),Reference (7),Spread (3),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Software and Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"2U, Inc.",NaN,Provider of course design and learning managem...,NaN,First lien senior secured loan,NaN,11.32 %,SOFR (M),6.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"AffiniPay Midco, LLC and AffiniPay Intermediat...",NaN,Payment processing solution provider,NaN,First lien senior secured loan,NaN,10.20 %,SOFR (A),5.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,First lien senior secured loan,NaN,10.39 %,SOFR (A),5.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,NaN,NaN,NaN,NaN,First lien senior secured loan,NaN,11.00 %,SOFR (Q),5.50 %,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557,NaN,NaN,NaN,NaN,Series A preferred stock,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
